In [ ]:
"""
Character‑level text generation with an LSTM RNN.
Trains on Shakespeare (tiny subset) and samples text with temperature control.
"""
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import pathlib
import os
import time

# 1. Load Shakespeare text (about 1 MB)
path = tf.keras.utils.get_file(
    "shakespeare.txt",
    "https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt",
)
text = pathlib.Path(path).read_text(encoding="utf‑8")
print(f"Corpus length: {len(text):,} characters")

# 2. Character vocabulary & vectorisation
chars = sorted(set(text))
print(f"Unique chars: {len(chars)}")
char2idx = {u: i for i, u in enumerate(chars)}
idx2char = np.array(chars)

def text_to_int(txt: str):
    return np.array([char2idx[c] for c in txt], dtype=np.int32)

encoded = text_to_int(text)

# Sequence length & dataset preparation
seq_len = 100
examples_per_epoch = len(text) // (seq_len + 1)

char_dataset = tf.data.Dataset.from_tensor_slices(encoded)
sequences = char_dataset.batch(seq_len + 1, drop_remainder=True)

def split_input_target(chunk):
    input_txt = chunk[:-1]
    target_txt = chunk[1:]
    return input_txt, target_txt

seq_ds = sequences.map(split_input_target)

BATCH_SIZE = 64
BUFFER_SIZE = 10000
ds = (
    seq_ds.shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.AUTOTUNE)
)

# 3. Build the RNN model
vocab_size = len(chars)
embedding_dim = 64
rnn_units = 256

model = tf.keras.Sequential(
    [
        layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[BATCH_SIZE, None]),
        layers.LSTM(rnn_units, return_sequences=True, stateful=True, recurrent_initializer="glorot_uniform"),
        layers.Dense(vocab_size),
    ]
)

# Loss (logits)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer="adam", loss=loss)

# 4. Train
EPOCHS = 10
model.fit(ds, epochs=EPOCHS)

# 5. Text generation (sampling one char at a time)

def generate_text(model, start_string: str, num_chars: int = 500, temperature: float = 1.0):
    """Generate text given a seed string and temperature."""
    # Rebuild model for batch_size=1
    temp_model = tf.keras.Sequential(
        [
            layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[1, None]),
            layers.LSTM(rnn_units, return_sequences=True, stateful=True, recurrent_initializer="glorot_uniform"),
            layers.Dense(vocab_size),
        ]
    )
    temp_model.set_weights(model.get_weights())

    input_eval = text_to_int(start_string)
    input_eval = tf.expand_dims(input_eval, 0)  # batch=1
    generated = []

    temp_model.reset_states()
    for _ in range(num_chars):
        preds = temp_model(input_eval)
        preds = preds[:, -1, :] / temperature  # focus on last step & scale by temperature
        predicted_id = tf.random.categorical(preds, num_samples=1)[-1, 0].numpy()

        input_eval = tf.expand_dims([predicted_id], 0)
        generated.append(idx2char[predicted_id])

    return start_string + "".join(generated)

# Sample with different temperatures
for temp in [0.5, 1.0, 1.5]:
    print("\n" + "=" * 20 + f" Temperature {temp}" + "=" * 20)
    print(generate_text(model, start_string="ROMEO:", temperature=temp, num_chars=300))

# -------------------------
# Temperature explanation:
# Lower values (e.g., 0.5) make the distribution sharper -> safer, repetitive text.
# Higher values (e.g., 1.5) flatten the distribution -> more randomness & creativity.
# -------------------------
